# Add AQI to data

This code will take a pandas df and return it with a new column added on, which is the AQI for the location and time the person was in that area.

In [5]:
# install HTTP requests package
%pip install requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: C:\Users\Admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [8]:
# imports
import pandas as pd
import numpy as np
import datetime as dt
import requests as req
import matplotlib as plt
import json

In [24]:
# load the 100,000 lines of my csv into a df (don't need that many for testing the script)
# show the first 10 rows, and the columns and their types
csv_data = pd.read_csv('../gsur23013.csv', nrows=1000000)
[(x, y) for (x, y) in zip(csv_data.columns, csv_data.dtypes)]
print(csv_data.head(10))

C:\Users\Admin\AppData\Local\Temp\ipykernel_22384\1515613150.py:3: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,18) have mixed types. Specify dtype option on import or set low_memory=False.
  csv_data = pd.read_csv('../gsur23013.csv', nrows=1000000)


                        stamp                   yaw                  pitch  \
0                          dt                     f                      f   
1  2023-08-28 11:47:13.245118  -0.03333548083901405    0.04052065312862396   
2  2023-08-28 11:47:13.345289   0.16603389382362366    0.05179822817444801   
3  2023-08-28 11:47:13.445459    0.9113951325416565   -0.03785695880651474   
4  2023-08-28 11:47:13.545630    0.9109325408935547  -0.024698100984096527   
5  2023-08-28 11:47:13.645800    0.9108432531356812   -0.02027300000190735   
6  2023-08-28 11:47:13.745970    0.9107893705368042  -0.018194498494267464   
7  2023-08-28 11:47:13.846141    0.9107804298400879  -0.016942808404564857   
8  2023-08-28 11:47:13.946312    0.9107078909873962  -0.016078751534223557   
9  2023-08-28 11:47:14.046482    0.9107396602630615  -0.015434675849974155   

                   roll        rotation_rate_x         rotation_rate_y  \
0                     f                      f                     

In [23]:
# Takes the given date and location and returns an AQI for that date and location. The AQI measure is *mainly* used with PM 2.5
# params
#   date: the date to retrieve the AQI for
#   latitude: the float representation of the latitude
#   longitude: the float representation of the longitude.
# returns
#   The AQI as a float
# Throws
#   Raises an HTTP Error if the request failed.
def GetAqi(date: dt.datetime, latitude: float, longitude: float) -> float:
    # this is the link to the historical API for AQI
    url = "https://www.airnowapi.org/aq/observation/latLong/historical?api_key={}&latitude={}&longitude={}&format=application/json&date={}T00-0000"
    
    # the api_key is stored in a config.json file, and kept off the github to avoid public exposure of your api_key for AirNow
    api_key = ""
    with open("../config.json", "r") as f:
        api_key = json.load(f)["api_key"]
    
    # formats the url with the right data, and sends it to AirNow
    response = req.get(url.format(api_key, latitude, longitude, date))
    
    # if the response is okay, it returns the AQI, if not, it raises an HTTP error
    if response.ok:
        return json.loads(response.json["AQI"])
    else:
        response.raise_for_status()

# definition
# params
# returns
# throws
def AddAQI(df: pd.DataFrame) -> pd.DataFrame:
    unique_dates = ""